# Predict The News Category Hackathon

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

## 1.Reading Data

In [4]:
train = pd.read_excel('/content/drive/MyDrive/Data/Participants_Data_News_category/Data_Train.xlsx')
test = pd.read_excel('/content/drive/MyDrive/Data/Participants_Data_News_category/Data_Test.xlsx')
sub = pd.read_excel('/content/drive/MyDrive/Data/Participants_Data_News_category/Sample_submission.xlsx')

train.shape, test.shape, sub.shape

((7628, 2), (2748, 1), (2748, 1))

## 2.Data PreProcessing

In [5]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [6]:
import re
import itertools
def remove_noise(text):
    # Make lowercase
    text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # Remove whitespaces
    text = text.apply(lambda x: " ".join(x.strip() for x in x.split()))
    
    # Convert to string
    text = text.astype(str)
        
    return text

In [7]:
train['STORY'] = remove_noise(train['STORY'])
test['STORY'] = remove_noise(test['STORY'])

## 3.Feature Engineering

In [8]:
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tt = WordPunctTokenizer()

In [10]:
cv = CountVectorizer(tokenizer=tt.tokenize, stop_words='english', ngram_range=(1,3))

cv.fit(train.STORY.tolist() + test.STORY.tolist())
train_cv = cv.transform(train.STORY)
test_cv = cv.transform(test.STORY)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [11]:
tf_idf = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1.0,smooth_idf=1.0,sublinear_tf=1.0,
            stop_words = 'english')

tf_idf.fit(train.STORY.tolist() + test.STORY.tolist())
train_tf = tf_idf.transform(train.STORY)
test_tf = tf_idf.transform(test.STORY)

## 3.Modeling

In [12]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC

In [14]:
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.SECTION)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.SECTION)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.SECTION)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.SECTION)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.SECTION)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.SECTION)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


SVC(C=1, class_weight='balanced', kernel='linear', probability=True)

## 4.Ensembing learning

In [16]:
from scipy.stats.mstats import mode

pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred = np.argmax(test_pred_prob, axis=1)


In [32]:
submission = pd.DataFrame()
submission['SECTION'] = pred.astype(int)
submission.to_excel("/content/drive/MyDrive/Data/Participants_Data_News_category/Predict The News Category Hackathon_submission.xlsx", index=None)
submission.head()

,SECTION
0,1
1,2
2,1
3,0
4,1
